In [1]:
from accelerate import notebook_launcher

from train import *

# os.environ['WANDB_NOTEBOOK_NAME'] = str(pathlib.Path().absolute())

init(False)

# write_basic_config(mixed_precision="no", save_location='~/.cache/huggingface/accelerate/default_config.yaml')
print(accelerate.accelerator.AcceleratorState._shared_state)


def launch(x, args):
    os.environ["NCCL_P2P_DISABLE"] = "1"
    os.environ["NCCL_IB_DISABLE"] = "1"
    notebook_launcher(x, args=args, num_processes=2)

wandb: Currently logged in as: mickjagger19. Use `wandb login --relogin` to force relogin
wandb: WARNING Disabling SSL verification.  Connections to this server are not verified and may be insecure!


{}


## Train VAE

In [2]:
vae_config = TrainingConfig()

# vae_config.train_batch_size = 11
vae_config.train_batch_size = 41
vae_config.num_epochs = 10
vae_config.vae_learning_rate = 1e-5
vae_config.save_model_epochs = 2
vae_config.save_image_epochs = 2

trainer = Trainer(
    vae_path='',
    generator_path='',
    config=vae_config,
    dataset_paths=['gtzan', 'fma']
)

os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"
vae_path = ''
# vae_path = ''
# from dataclasses import asdict
# 
# print(asdict(vae_config))

accelerate_path = ''
launch(
    trainer.train_vae, (True, None))

loading 
Launching training on 2 GPUs.


wandb: WARNING Disabling SSL verification.  Connections to this server are not verified and may be insecure!


Epoch 0:   0%|          | 0/45 [00:00<?, ?it/s]

RuntimeError: An issue was found when launching the training: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/root/miniconda3/envs/remixer/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", line 74, in _wrap
    fn(i, *args)
  File "/root/miniconda3/envs/remixer/lib/python3.9/site-packages/accelerate/utils/launch.py", line 562, in __call__
    self.launcher(*args)
  File "/root/autodl-tmp/remixer/training/train.py", line 98, in train_vae
    for step, batch in enumerate(dataloader):
  File "/root/miniconda3/envs/remixer/lib/python3.9/site-packages/accelerate/data_loader.py", line 448, in __iter__
    current_batch = next(dataloader_iter)
  File "/root/miniconda3/envs/remixer/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
  File "/root/miniconda3/envs/remixer/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 674, in _next_data
    data = self._datase


## Train Diffusion

In [ ]:
diffusion_config = TrainingConfig()
diffusion_config.train_batch_size = 3
diffusion_config.num_epochs = 4
diffusion_config.latent_channels = 4
diffusion_config.save_model_epochs = 2
diffusion_config.save_image_epochs = 2

trainer = Trainer(
    vae_path='',
    generator_path='',
    config=diffusion_config,
    dataset_paths=['gtzan', 'fma']
)

launch(trainer.train_diffusion, ())

## Test

In [ ]:
config = TrainingConfig()
from test_model import *

vae_path = '/root/autodl-tmp/remixer/training/artifacts/atomic-armadillo-16/AutoencoderKL/model_epoch_7.pth'
generator_path = '/root/autodl-tmp/remixer/training/artifacts/celestial-firebrand-7/VQGenerator/model_epoch_1.pth'
scheduler_path = '/root/autodl-tmp/remixer/training/artifacts/celestial-firebrand-7/scheduler_config.json'

# vae_path = '/root/autodl-tmp/remixer/training/artifacts/kind-spaceship-380/AutoencoderKL/kind-spaceship-380_models_epoch_9.pth'
config.num_inference_steps = 999
datas = [generate_from_genre(config, genre=None,
                             vae_path=vae_path,
                             generator_path=generator_path,
                             scheduler_path=scheduler_path)
         for _ in range(3)]

make_grid([img for output, img in datas])